In [1]:
#Import necessary libraries
import os
import ee
import geemap
import ipyleaflet
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import statsmodels.api as sm
import pandas as pd
from IPython.display import HTML, display
import random
import json
import time
num_seed=30
random.seed(num_seed)

In [2]:
#Initialize earth engine
ee.Initialize()


In [3]:
#Define functions for mapping MapBiomas and simplifying the legend
coverage_palette =  ['ffffff', '129912', '1f4423', '006400', '00ff00', '687537', '76a5af', '29eee4', 
                     '77a605', '935132', 'bbfcac', '45c2a5', 'b8af4f', 'f1c232', 'ffffb2', 'ffd966', 
                     'f6b26b', 'f99f40', 'e974ed', 'd5a6bd', 'c27ba0', 'fff3bf', 'ea9999', 'dd7e6b', 
                     'aa0000', 'ff99ff', '0000ff', 'd5d5e5', 'dd497f', 'b2ae7c', 'af2a2a', '8a2be2', 
                     '968c46', '0000ff', '4fd3ff']


simple_palette = ['129912','BBFCAC','FFFFB2','EA9999','0000FF','D5D5E5']
statesViz = {'min': 0, 'max': 34, 'palette': coverage_palette};
simpleStatesViz = {'min': 1, 'max': 6, 'palette': simple_palette};

change_detection_palette = ['df07b5','0741df']
changeDetectionViz = {'min': 0, 'max': 1, 'palette': change_detection_palette};

#Load in mapbiomas
mapbiomas_states=ee.Image('projects/mapbiomas-workspace/public/collection4_1/mapbiomas_collection41_integration_v1')
states_mask = mapbiomas_states.mask()

#Define function to convert hierarchical legend to simplest form
def simplify_legend(bandName):
    simplify = mapbiomas_states.expression(
        '(b0 >=1)  && (b0<10) ? 1 :'+
        '((b0>=10) && (b0<14)) || (b0==32) || (b0==29) ? 2 :'+
        '((b0>=18) && (b0<22)) || ((b0>=14)&&(b0<16)) ? 3 :'+
        '((b0>=22) && (b0<26)) || (b0==30) ? 4 :'+
        '(b0==26) || (b0==33) || (b0==31) ? 5 : 6', 
        {
          'b0': mapbiomas_states.select([bandName])
        })
    simplify = simplify.select(['constant'],[bandName])
    return simplify

#Select bands we are interested in
bandList = ['classification_1985', 'classification_1986', 'classification_1987', 'classification_1988', 
             'classification_1989', 'classification_1990', 'classification_1991', 'classification_1992', 
             'classification_1993', 'classification_1994', 'classification_1995', 'classification_1996', 
             'classification_1997', 'classification_1998', 'classification_1999', 'classification_2000', 
             'classification_2001', 'classification_2002', 'classification_2003', 'classification_2004', 
             'classification_2005', 'classification_2006', 'classification_2007', 'classification_2008', 
             'classification_2009', 'classification_2010', 'classification_2011', 'classification_2012', 
             'classification_2013', 'classification_2014', 'classification_2015', 'classification_2016', 
             'classification_2017', 'classification_2018']
bandsEEList = ee.List(bandList) 
states_simple = ee.ImageCollection(bandsEEList.map(simplify_legend)).toBands()
states_simple = states_simple.select(states_simple.bandNames(),bandsEEList)
states_simple = states_simple.updateMask(states_mask)
states_simple = states_simple.set(ee.Dictionary({'min_value':1,'max_value':5}))
#states_simple is now an iamge where each band corresponds to the land cover class for the band name year

#Map one year to check it out!
Map1 = geemap.Map(center=[-9,-51], zoom=4)
Map1.addLayer(mapbiomas_states.select('classification_2018'),statesViz,name='Original MapBiomas')
Map1.addLayer(states_simple.select('classification_2018'),simpleStatesViz,name='Simplified MapBiomas')
display(Map1)


Map(center=[-9, -51], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=F…

In [4]:
#Save 30 meter projection
projection_30m = mapbiomas_states.projection()


In [5]:
#Convert long band names to short band names
intBandNames = ['1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', 
             '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',  '2005', '2006', '2007', '2008', 
             '2009', '2010', '2011', '2012','2013', '2014', '2015', '2016', '2017', '2018']

states_simple = states_simple.select(bandList,intBandNames)


In [6]:

#Image bands must be ordered by increasing years
def get_year_stack_image_collection(image, band_names, band_indices=[-1,0,1]):
    '''
    Function returns image collection of images where each band is taken from the band_indices. If inputted bands do
                do not follow the band indices, that image will not be returned. 
                For example if one band index is less than 0, an image for the first band will not be returned
                because there is not a band corresponding to that index.
    Inputs:
        image: image where each band represents the land cover classification for a year, bands ordered by 
                increasing years
        band_names: list of band names in the image
        band_indices: list of indices you want to collect from the image, the default [-1,0,1] will return an 
                image collection where each image will have the bands [previous year, current year, following year]
    Returns:
        out_image_list: an image collection where each image corresponds to a band in band_names, where the bands 
                of the image correspond to the band_indices input
                
    Example:
        Inputs:
            image = image of land cover classification for years [1986,1987,1988,1989]
            band_names = [1986,1987,1988,1989]
            band_indices = [-1,0,1]
        Returns:
            out_image_list = image collection with the following images:
                image 1: bands: [1986,1987,1988], property {'OriginalBand': 1987}
                image 2: bands: [1987,1988,1989], property {'OriginalBand': 1988}
            (an image for 1986 is not included because there is not a year before 1986,
             and an image for 1989 is not included because there is not a year after 1989)
    '''
    out_image_list = []
    for i,band_name in enumerate(band_names):
        #indices = i_
        if all(np.array([int(i+x) for x in band_indices])>=0):
            try:
                band_list = [band_names[i+x] for x in band_indices]
                out_image = ee.Image.cat(image.select(band_list))
                out_image = out_image.set(ee.Dictionary({'OriginalBand':band_name}))
                out_image_list.append(out_image)
            except:
                None
    
    return ee.ImageCollection(out_image_list)


In [23]:
#Functions for binary land cover change properties
def lc_one_change(image):
    '''
    Determines if there was one change occurance from year i to year i+1. Returns an image with values:
    1 if state(i) != state(i+1)
    0 if state(i) == state(i+1)
    '''
    band_names = image.bandNames()
    out_image = image.select([band_names.get(0)]).neq(image.select([band_names.get(1)]))
    out_image = out_image.select(out_image.bandNames(),[band_names.get(0)])
    out_image = out_image.set(ee.Dictionary({'OriginalBand':band_names.get(0)}))
    return out_image

def lc_no_change(image):
    '''
    Determines if there was no change occurance from year i to year i+1. Returns an image with values:
    1 if state(i) != state(i+1)
    0 if state(i) == state(i+1)
    '''
    band_names = image.bandNames()
    out_image = image.select([band_names.get(0)]).eq(image.select([band_names.get(1)]))
    out_image = out_image.select(out_image.bandNames(),[band_names.get(0)])
    out_image = out_image.set(ee.Dictionary({'OriginalBand':band_names.get(0)}))
    return out_image

def lc_reverse(image):
    '''
    Determines if change that occured from i to i+1 reversed back to state i in i+2
    1 if state(i) != state(i+1) and state(i) == state(i+2)
    0 otherwise
    '''
    band_names = image.bandNames()
    current_year = image.select([band_names.get(0)])
    next_year = image.select([band_names.get(1)])
    next_next_year = image.select([band_names.get(2)])
    
    returnback = current_year.eq(next_next_year)
    changed = current_year.neq(next_year)
    out_image = returnback.bitwise_and(changed)
    out_image = out_image.select(out_image.bandNames(),[band_names.get(0)])
    out_image = out_image.set(ee.Dictionary({'OriginalBand':band_names.get(0)}))
    return out_image

def lc_change_to_another(image):
    '''
    Determines if change occured from i to i+1 and change occured in i+1 to i+2 where state(i)!=state(i+2)
    1 if state(i) != state(i+1) and state(i) != state(i+2) and state(i+1) != state(i+2)
    0 otherwise
    '''
    band_names = image.bandNames()
    current_year = image.select([band_names.get(0)])
    next_year = image.select([band_names.get(1)])
    next_next_year = image.select([band_names.get(2)])
    
    changed = current_year.neq(next_year)
    changed_again = next_year.neq(next_next_year)
    not_reversed = current_year.neq(next_next_year)
    
    out_image = changed.bitwise_and(changed_again.bitwise_and(not_reversed))
    out_image = out_image.select(out_image.bandNames(),[band_names.get(0)])
    out_image = out_image.set(ee.Dictionary({'OriginalBand':band_names.get(0)}))
    return out_image

def lc_consistent_change_one_year(image):
    '''
    Determines if change that occured from i to i+1 stayed in i+2
    1 if state(i) != state(i+1) and state(i+1) == state(i+2)
    0 otherwise
    '''
    band_names = image.bandNames()
    current_year = image.select([band_names.get(0)])
    next_year = image.select([band_names.get(1)])
    next_next_year = image.select([band_names.get(2)])
    
    changed = current_year.neq(next_year)
    stayed = next_year.eq(next_next_year)
    
    out_image = changed.bitwise_and(stayed)
    out_image = out_image.select(out_image.bandNames(),[band_names.get(0)])
    out_image = out_image.set(ee.Dictionary({'OriginalBand':band_names.get(0)}))
    return out_image

def lc_consistent_change_two_years(image):
    '''
    Determines if change that occured from i to i+1 stayed in i+2 and i+3
    1 if state(i) != state(i+1) and state(i+1) == state(i+2) and state(i+1) == state(i+3)
    0 otherwise
    '''
    band_names = image.bandNames()
    current_year = image.select([band_names.get(0)])
    next_year = image.select([band_names.get(1)])
    next_next_year = image.select([band_names.get(2)])
    next_next_next_year = image.select([band_names.get(2)])
    
    changed = current_year.neq(next_year)
    stayed = next_year.eq(next_next_year)
    stayed_again = next_year.eq(next_next_next_year)
    
    out_image = changed.bitwise_and(stayed.bitwise_and(stayed_again))
    out_image = out_image.select(out_image.bandNames(),[band_names.get(0)])
    out_image = out_image.set(ee.Dictionary({'OriginalBand':band_names.get(0)}))
    return out_image

def lc_year_after(image):
    '''
    Returns land cover class for following year
    '''
    band_names = image.bandNames()
    current_year = image.select([band_names.get(0)])
    next_year = image.select([band_names.get(1)])
    out_image = next_year.select(next_year.bandNames(),[band_names.get(0)])
    out_image = out_image.set(ee.Dictionary({'OriginalBand':band_names.get(0)}))
    return out_image



In [30]:
#Apply land cover change functions to images, first returns an image collection then converted to image
#where each band represents one year
lc_one_change_col = get_year_stack_image_collection(states_simple,states_simple.bandNames().getInfo(), band_indices=[0,1])
lc_one_change_col = lc_one_change_col.map(lc_one_change)
lc_one_change_image = lc_one_change_col.toBands()
lc_one_change_image = lc_one_change_image.select(lc_one_change_image.bandNames(),lc_one_change_col.aggregate_array('OriginalBand'))

lc_no_change_col = get_year_stack_image_collection(states_simple,states_simple.bandNames().getInfo(), band_indices=[0,1])
lc_no_change_col = lc_no_change_col.map(lc_no_change)
lc_no_change_image = lc_no_change_col.toBands()
lc_no_change_image = lc_no_change_image.select(lc_no_change_image.bandNames(),lc_no_change_col.aggregate_array('OriginalBand'))

lc_reversed_col = get_year_stack_image_collection(states_simple,states_simple.bandNames().getInfo(), band_indices=[0,1,2])
lc_reversed_col = lc_reversed_col.map(lc_reverse)
lc_reversed_image = lc_reversed_col.toBands()
lc_reversed_image = lc_reversed_image.select(lc_reversed_image.bandNames(), lc_reversed_col.aggregate_array('OriginalBand'))

lc_changed_to_another_col = get_year_stack_image_collection(states_simple,states_simple.bandNames().getInfo(), band_indices=[0,1,2])
lc_changed_to_another_col = lc_changed_to_another_col.map(lc_change_to_another)
lc_changed_to_another_image = lc_changed_to_another_col.toBands()
lc_changed_to_another_image = lc_changed_to_another_image.select(lc_changed_to_another_image.bandNames(), lc_changed_to_another_col.aggregate_array('OriginalBand'))

lc_consistent_change_one_year_col = get_year_stack_image_collection(states_simple,states_simple.bandNames().getInfo(), band_indices=[0,1,2])
lc_consistent_change_one_year_col = lc_consistent_change_one_year_col.map(lc_consistent_change_one_year)
lc_consistent_change_one_year_image = lc_consistent_change_one_year_col.toBands()
lc_consistent_change_one_year_image = lc_consistent_change_one_year_image.select(lc_consistent_change_one_year_image.bandNames(), lc_consistent_change_one_year_col.aggregate_array('OriginalBand'))

lc_consistent_change_two_years_col = get_year_stack_image_collection(states_simple,states_simple.bandNames().getInfo(), band_indices=[0,1,2,3])
lc_consistent_change_two_years_col = lc_consistent_change_two_years_col.map(lc_consistent_change_two_years)
lc_consistent_change_two_years_image = lc_consistent_change_two_years_col.toBands()
lc_consistent_change_two_years_image = lc_consistent_change_two_years_image.select(lc_consistent_change_two_years_image.bandNames(), lc_consistent_change_two_years_col.aggregate_array('OriginalBand'))

lc_year_after_col = get_year_stack_image_collection(states_simple,states_simple.bandNames().getInfo(), band_indices=[0,1])
lc_year_after_col = lc_year_after_col.map(lc_year_after)
lc_year_after_image = lc_year_after_col.toBands()
lc_year_after_image = lc_year_after_image.select(lc_year_after_image.bandNames(), lc_year_after_col.aggregate_array('OriginalBand'))


In [31]:
#Map consistent change layer for one year to test if it works
Map2 = geemap.Map(center=[-9,-51], zoom=4)
Map2.addLayer(lc_consistent_change_two_years_image.select(['1990']),changeDetectionViz,name='1990 Consistent Change')
Map2.addLayer(states_simple.select(['1990']),simpleStatesViz,name='1990')
Map2.addLayer(states_simple.select(['1991']),simpleStatesViz,name='1991')
Map2.addLayer(states_simple.select(['1992']),simpleStatesViz,name='1992')
display(Map2)




Map(center=[-9, -51], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=F…

In [32]:
#Define kernel for neighborhood
kernel = ee.Kernel.fixed(3,3,
                         [[1,1,1],
                          [1,0,1],
                          [1,1,1]]
                          ,1,1)

#Other kernel options commented below:
#kernel = ee.Kernel.gaussian(radius=1000, units='meters', sigma=1000)

#kernel = ee.Kernel.circle(radius=17, units='pixels')
#kernel = ee.Kernel.square(radius=1.5, units='pixels')


In [33]:
#Get neighborhood information on pixels
states_lc_no_change_neighbors = lc_no_change_image.convolve(kernel)
states_lc_one_change_neighbors = lc_one_change_image.convolve(kernel)
states_lc_reverse_neighbors = lc_reversed_image.convolve(kernel)
states_lc_change_to_another_neighbors = lc_changed_to_another_image.convolve(kernel)
states_lc_consistent_change_one_year_neighbors = lc_consistent_change_one_year_image.convolve(kernel)
states_lc_consistent_change_two_years_neighbors = lc_consistent_change_two_years_image.convolve(kernel)


In [46]:
#Define list of images and the corresponding names (which will be used as column names) to sample for covariates
image_list = [states_simple, 
              lc_year_after_image,
              states_lc_no_change_neighbors,
              states_lc_one_change_neighbors,
              states_lc_reverse_neighbors,
              states_lc_change_to_another_neighbors,
              states_lc_consistent_change_one_year_neighbors,
              states_lc_consistent_change_two_years_neighbors]

image_name_list = ['current_state',
                   'year_after_state',
                   'no_lc_change_neighbors',
                   'one_lc_change_neighbors',
                   'lc_reverse_neighbors',
                   'lc_change_to_another_neighbors',
                   'lc_consistent_change_one_year_neighbors',
                   'lc_consistent_change_two_years_neighbors']

#Reproject images to original reprojection
image_list = [x.reproject(projection_30m) for x in image_list]



In [47]:
#Functions I've written to try to do this sampling

#Function to convert feature collection to pandas dataframe
def get_dataframe_from_feature_collection(feature_collection, property_names):
    df = pd.DataFrame()
    for property_name in property_names:
        property_values = feature_collection.aggregate_array(property_name).getInfo()
        df[property_name] = property_values
    return df

#Function to convert pandas dataframe to feature collection
def convert_points_df_to_feature_collection(df,projection='EPSG:4326',lat_name='latitude',lon_name='longitude'):
    feature_collection_list = []
    for i,row in df.iterrows():
        geometry = ee.Geometry.Point([row[lon_name],row[lat_name]])#,projection)
        row_dict = row.to_dict()
        row_feature = ee.Feature(geometry,row_dict)
        feature_collection_list.append(row_feature)
    return ee.FeatureCollection(feature_collection_list)

#Function to convert pandas dataframe to feature collection
def convert_point_df_to_feature(series,projection='EPSG:4326',lat_name='latitude',lon_name='longitude'):
    geometry = ee.Geometry.Point([series[lon_name],series[lat_name]])#,projection)
    row_dict = series.to_dict()
    row_feature = ee.Feature(geometry,row_dict)
    return row_feature

def getSampleImageData(image, sampleBandPoints):
    sampleImageData = image.reduceRegions(
            collection=sampleBandPoints,
            reducer=ee.Reducer.first()
            )
    return sampleImageData.aggregate_array('first')

def getSampleImageDataFromPoint(image, sampleBandPoint):
    sampleImageData = image.reduceRegion(
            geometry=sampleBandPoint.geometry(),
            reducer=ee.Reducer.first()
            )
    return sampleImageData.get(image.bandNames().get(0))


In [48]:
# Functions Francis used in his notebook (permament_change_metrics) copied here
# def getSampleBandPoints(image, region, **kwargs):
#     dargs = {
#         'numPoints':100,
#         'region':region,
#         'classValues':[0,1], 
#         'classPoints':[0,100],
#         'seed':794274,
#         'geometries':True,
#     }
#     dargs.update(kwargs)
#     bandSamples = image.bandNames().map(lambda b:
#         image.select(ee.String(b)).stratifiedSample(
#             **dargs),
#         )
#     return bandSamples

# def getSampleBandData(image, sampleBandPoints):
#     idxIter = ee.List.sequence(0, count=sampleBandPoints.length())
#     sampleBandData = idxIter.map(lambda i:
#         image.select(ee.Number(i)).reproject(image.projection()).reduceRegions(
#             collection=sampleBandPoints.get(ee.Number(i)),
#             reducer=ee.Reducer.first()
#         ))
#     return sampleBandDataToList(sampleBandData)

# def sampleBandDataToList(sampleData):
#     return ee.FeatureCollection(sampleData).flatten().aggregate_array('first')

# sample_points = getSampleBandPoints(changed_to, geometry)

# res = ee.Dictionary({
#     'next_is_same': getSampleBandData(changed_and_stayed, sample_points),
#     'neighbors': getSampleBandData(lc_neighbors, sample_points),
#     'to_neighbors': getSampleBandData(changed_to_neighbors, sample_points),
#     'away_neighbors': getSampleBandData(changed_away_neighbors, sample_points)
# }).getInfo()

In [49]:
#Read in training data
training_points_url = 'https://raw.githubusercontent.com/kristinelister/WRI-NGS-DynamicWorld/master/MapBiomas_ChangeDetection/TrainingPoints/Sample_Points_1000_wchange.csv'
training_points = pd.read_csv(training_points_url)
training_points_fc = convert_points_df_to_feature_collection(training_points)
display(training_points)


,consistent_change,latitude,longitude,year
0,0,-20.323530,-41.074882,2014
1,0,-9.748832,-63.887330,2015
2,0,-3.562314,-60.710258,2015
3,0,-3.482274,-42.444723,2014
4,0,-18.723002,-56.919278,2015
...,...,...,...,...
1995,1,-3.548839,-45.688091,2012
1996,1,-5.488122,-35.278323,1990
1997,1,-14.634499,-41.844828,2000
1998,1,-3.396036,-39.927644,1992


In [50]:
#ALEX - THIS IS MY FIRST STAB AT DOING THIS - TAKES A REALLY LONG TIME!!
#Sample covariates from image list defined above at our training point locations
training_points_wcovar = training_points.copy()

#Iterate through each row of the dataframe
for i,row in training_points_wcovar.iterrows():
    #Stopped it at the first row to check it out
    if i==0:
        #Convert row to EE feature
        row_fc = convert_point_df_to_feature(row)
        #Iterate through image list
        for zippy in zip(image_list,image_name_list):
            image = zippy[0]
            image_name = zippy[1]
            #Reproject image to 30 meter resolution
            image = image.select(str(int(row['year'])))
            #Get value of image at point location
            output = getSampleImageDataFromPoint(image, row_fc).getInfo()
            #Save to dataframe
            training_points_wcovar.at[i,image_name] = output
            training_points_wcovar.to_csv('/Users/kristine/Downloads/NewSampledPoints_wcovariates_2.csv',index=False)


            
display(training_points_wcovar)


,consistent_change,latitude,longitude,year,current_state,year_after_state,no_lc_change_neighbors,one_lc_change_neighbors,lc_reverse_neighbors,lc_change_to_another_neighbors,lc_consistent_change_one_year_neighbors,lc_consistent_change_two_years_neighbors
0,0,-20.323530,-41.074882,2014,1.0,3.0,8.0,0.0,0.0,0.0,0.0,0.0
1,0,-9.748832,-63.887330,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,-3.562314,-60.710258,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,-3.482274,-42.444723,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,-18.723002,-56.919278,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,-3.548839,-45.688091,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,1,-5.488122,-35.278323,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,1,-14.634499,-41.844828,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,1,-3.396036,-39.927644,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# #Map one row to check it out (last row in DF, replace center with coordinates of row_fc)
# Map5 = geemap.Map(center=[-20.323530,-41.074882], zoom=12)
# Map5.addLayer(states_simple.select('2014'),simpleStatesViz,name='LC 2014')
# Map5.addLayer(states_simple.select('2015'),simpleStatesViz,name='LC 2015')
# Map5.addLayer(row_fc)
# display(Map5)

Map(center=[-20.32353, -41.074882], controls=(WidgetControl(options=['position'], widget=HBox(children=(Toggle…